%load_ext autoreload
%autoreload 2

In [11]:
import numpy as np
# some handy functions to use along widgets
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets

In [12]:
import sys

# setting path
sys.path.append('../client')
import serving_client
import game_Client 

In [13]:
#Here I am instanciating the classes from game_client and serving_client

pinger = game_Client.gameClient(ip="127.0.0.1",port=6758)
serving = serving_client.AppClient(ip="127.0.0.1",port=6758)

In [14]:
game = pinger.pingGame('2021020329')

In [20]:
serving.predict(game,'2021020329')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [16]:
#Input text for the model
text1 = widgets.Text(
       value='workspaceTest',
       description='workspace',)
text2 = widgets.Text(
       value='modelTest',
       description='model',)
text3 = widgets.Text(
       value='versionTest',
       description='version',)

#Button1 will load the model of choice
button1 = widgets.Button(description='Load Model')

#Input text for gameID
text4 = widgets.Text(
       value='2021020329',
       description='gameID',) 

#Button2 will ping the game
button2 = widgets.Button(description='Ping Game')
out = widgets.Output()

#Here will be stored the expected goals 
XGt1=[]
XGt2=[]

def on_button1_clicked(_):
    #This function activates when button 1 is pushed
      with out:
          clear_output()  
          #download_registry_model(self, workspace: str, model: str, version: str)
          print("Model '",text1.value, "' Version '",text2.value,"' Workspace '",text3.value," has been uploaded")
            
def on_button2_clicked(_):
      # This function activates when button 2 is pushed 
      with out:
          clear_output()
            
          #Ping game for a given gameID
          game = pinger.pingGame(str(text4.value))
            
          #Get current period and time left for that period  
          period = game.iloc[-1]["period"]
          time = str(game.iloc[-1]["periodTime"]).split(":")
          minuteLeft = 19-int(time[0])
          secondLeft=60-int(time[1])
          timeLeft= str(minuteLeft)+":"+str(secondLeft)
        
          #Get teams and homeOrAway
          teams=list(game["teamOfShooter"])
          homeOrAway=list(game["homeOrAway"])
          teams = list(dict.fromkeys(teams))
          homeOrAway= list(dict.fromkeys(homeOrAway))
           
          #Separate dataframes for each team, and get their corresponding XG
          gamet1=game.loc[game["teamOfShooter"]==teams[0]]
          gamet2=game.loc[game["teamOfShooter"]==teams[1]]
          XGt1.extend(serving.predict(gamet1))
          XGt2.extend(serving.predict(gamet2))
          #sum of XG
          sumt1 = sum(XGt1)
          sumt2= sum(XGt2)
        
          #Output to print 
          print("period:",period)
          print("time left:",timeLeft)
          print(homeOrAway[0],"team and expected score:",teams[0],sumt1)
          print(homeOrAway[1],"team and expected score:",teams[1],sumt2)


        

#Associate buttons to functions
button1.on_click(on_button1_clicked)
button2.on_click(on_button2_clicked)
#Widgets that will be displayed
widgets.VBox([text1,text2,text3,button1,text4,button2,out])

    def ping_game(game_id, idx, other) -> new_events, new_idx, new_other
    
    def predict(self, X: pd.DataFrame) -> pd.DataFrame:
        """
        Formats the inputs into an appropriate payload for a POST request, and queries the
        prediction service. Retrieves the response from the server, and processes it back into a
        dataframe that corresponds index-wise to the input dataframe.
        
        Args:
            X (Dataframe): Input dataframe to submit to the prediction service.
        """

        raise NotImplementedError("TODO: implement this function")

    def logs(self) -> dict:
        """Get server logs"""

        raise NotImplementedError("TODO: implement this function")

    def download_registry_model(self, workspace: str, model: str, version: str) -> dict:
        """
        Triggers a "model swap" in the service; the workspace, model, and model version are
        specified and the service looks for this model in the model registry and tries to
        download it. 

        See more here:

            https://www.comet.ml/docs/python-sdk/API/#apidownload_registry_model
        
        Args:
            workspace (str): The Comet ML workspace
            model (str): The model in the Comet ML registry to download
            version (str): The model version to download
        """

        raise NotImplementedError("TODO: implement this function")